<a href="https://colab.research.google.com/github/shashiwadana/Answerable/blob/main/GPT_responses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install --upgrade openai

In [ ]:
import pandas as pd
file = '/content/Annotation - updated.xlsx'

df = pd.read_excel(file, sheet_name = 'Annotation 1.0', header = 0)#, nrows=3015) # top 3 rows are summary calculation
# df = df.drop(df.index[[0,1,2]])

nrows_include = 3349
df = df.drop(df.index[nrows_include:]) # include till

df1 = pd.read_excel(file, sheet_name = 'Annotation 2.0', header = 0)#, nrows=3015) # top 3 rows are summary calculation
# df1 = df1.drop(df1.index[[0,1,2]]) # removing rows 1-3 as they are stats

# Replacing column 'Label (1 - age related) \nAnnotator 1-V2' to 'Label (1 - age related)\nAnnotator 1' to make it uniform with df
df1 = df1.drop('Label (1 - age related)\nAnnotator 1', axis=1)
df1.rename(columns={'Label (1 - age related) \nAnnotator 1-V2': 'Label (1 - age related)\nAnnotator 1'}, inplace=True)

df = pd.concat([df, df1], axis=0)
df = df.reset_index(drop=True)
# Combining the annotations from 3 annotators
df['labels'] = df[['Label (1 - age related)\nAnnotator 1',
   'Label (1 - age related) \nAnnotator  2',
   'Label (1 - age related) \nAnnotator  3 (In case of disagreements)']].mode(axis=1)[0]

print('Total num of reviews:', len(df))

Total num of reviews: 4163


In [ ]:
# df = df.loc[3000:]#lock.loc[0:3]
df = df.reset_index(drop = True)
df

,Review_id,app_name,RegEx,Content,Label (1 - age related)\nAnnotator 1,Label (1 - age related) \nAnnotator 2,Label (1 - age related) \nAnnotator 3 (In case of disagreements),Unnamed: 7,Comments,Disagreement,labels
0,1.0,paypal,kids,The easiest way to send money to my kids. I lo...,0.0,0.0,NaN,NaN,NaN,0.0,0.0
1,2.0,youtube,children,I am really disappointed I can't remove ads fr...,1.0,1.0,NaN,NaN,NaN,0.0,1.0
2,3.0,twitter,age,"Excuse me, i was send a message one month ago,...",1.0,1.0,NaN,NaN,NaN,0.0,1.0
3,4.0,tiktok,kids,Bruh nobody wants to see you hitting a blunt a...,0.0,0.0,NaN,NaN,NaN,0.0,0.0
4,5.0,messenger,old,I use this app alot and used to be great. But ...,0.0,0.0,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
4158,5810.0,creditkarma,retired,retired disabled veteran,0.0,0.0,NaN,NaN,NaN,0.0,0.0
4159,5811.0,google,all ages,Yndjjjwjfbe fuwhfx decided to . If you have 88...,0.0,0.0,NaN,NaN,NaN,0.0,0.0
4160,5812.0,youtubemusic,retired,**PLEASE ADD THE SHAZAM FEATURE** I am NOT a f...,0.0,0.0,NaN,NaN,NaN,0.0,0.0
4161,5813.0,googleduo,grandparents,"Awesome app. Super easy to use, quality is pre...",1.0,0.0,1.0,NaN,NaN,1.0,1.0


# Without context
First, classifying the reviews from the GPT model. A prompt is created but without any context for the tesk.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from openai import OpenAI
import time
import openai

# optional; defaults to `os.environ['OPENAI_API_KEY']`
openai.api_key = ''

prompt = "You will be provided with some mobile app user reviews. Your task is to classify it as YES if it discusses any \
age related concern/requirement/issues from the mobile app user perspective."

def gpt_review(review):
    completion = openai.chat.completions.create(
      model="gpt-3.5-turbo-0125", #"gpt-4",
      messages=[
          {
            "role": "system",
            "content": prompt,
          },
          {
              "role": "user",
              "content": review,
          },
      ],
      temperature=0.1,
      max_tokens=64,
      top_p=0.1
    )
    # print(completion.choices[0].message.content)
    return completion.choices[0].message.content

In [ ]:
from tqdm import tqdm
import time

gpt_response_no_context = []
for index, row in tqdm(df.iterrows()):
    # print(row["Content"])
    gpt_output = gpt_review(row["Content"])
    # print(gpt_output)
    # time.sleep(1)
    if "YES" in gpt_output:
        gpt_response_no_context.append(1)
    elif "NO" in gpt_output:
        gpt_response_no_context.append(0)
    else:
        gpt_response_no_context.append("Invalid")


4160it [51:33,  1.34it/s] 


In [ ]:
df["gpt_pred_no_context"] = gpt_response_no_context

In [ ]:
df.to_excel("/content/drive/MyDrive/age_gpt_analysis/gpt_generated_labels.xlsx", index=False, sheet_name='gpt_pred_no_context')

# With context
Now adding a detailed context to help GPT to understand the task better. A detailed prompt is added along with few example reviews with their age-related category.

In [ ]:
context_prompt = "You will be provided with some mobile app user reviews. Your task is to classify it as YES if it discusses any \
age related concern/requirement/issues froma the mobile app user perspective."

example_1 = "I love it, I can see my beautiful grandchildren anytime on this app. I absolutely would recommend it's to everyone."
ans_1 = "YES"

example_2 = "This app is ok. I like the features but it keeps displaying the comment about carpooling when Im driving and I CANNOT see \
MY SCREEN when this happens! Why does it keep reappearing. I'm not interested in carpooling or sharing a ride. I'm retired not working, \
volunteering at different places and not everyday. I get very annoyed when this happens. How can I disable this carpool prompt permanently \
 to keep my stress down or I will delete this Waze app?"
ans_2 = "NO"

example_3 = "I've been using insta for atleast 9-10 years now it's saying I'm under age to use this platform."
ans_3 = "YES"

example_4 = "Installation was long, login process hampered by some issue with my established email. I have an account but was unable to \
get past entering my email address. It simply defaults back to my age. App issue I would imagine. Oh well."
ans_4 = "NO"

In [ ]:
from openai import OpenAI
import time
import openai

# optional; defaults to `os.environ['OPENAI_API_KEY']`
openai.api_key = ''

def gpt_completion_context(review):
      completion = openai.chat.completions.create(
      model="gpt-3.5-turbo-0125", #"gpt-4",
      messages=[
          { "role": "system", "content": context_prompt, },
          { "role": "user", "content": example_1, },
          { "role": "assistant", "content": ans_1, },
          { "role": "user", "content": example_2, },
          { "role": "assistant", "content": ans_2, },
          { "role": "user", "content": example_3, },
          { "role": "assistant", "content": ans_3, },
          { "role": "user", "content": example_4, },
          { "role": "assistant", "content": ans_4, },
          { "role": "user", "content": review, },
      ],
      temperature=0.1,
      max_tokens=64,
      top_p=0.1
      )
      # print(completion.choices[0].message.content)
      return completion.choices[0].message.content

In [ ]:
from tqdm import tqdm
import time
import numpy as np

gpt_response_context = np.zeros([len(df)])
for index, row in tqdm(df.iterrows()):
    # print('----\n',index, row["Content"])
    gpt_output = gpt_completion_context(row["Content"])
    # print(gpt_output)
    # time.sleep(0.2)
    if "YES" in gpt_output:
        gpt_response_context[index] = 1
    elif "NO" in gpt_output:
        gpt_response_context[index] = 0
    else:
        gpt_response_context[index] = -1


1163it [24:16,  1.25s/it] 


In [ ]:
len(gpt_response_context)

1163

In [ ]:
df["gpt_pred_context"] = pd.Series(gpt_response_context)

In [ ]:
df.to_excel("/content/drive/MyDrive/age_gpt_analysis/gpt_generated_labels.xlsx", index=False, sheet_name='gpt_pred_context')

# Performance Stats

In [ ]:
# Code to compare the GPT labels with the ground truth labels
from sklearn import metrics
from sklearn.metrics import roc_auc_score,roc_curve
def scores(y_test, predictions):
    precision = metrics.precision_score(y_test, predictions)
    recall = metrics.recall_score(y_test, predictions)
    f1 = metrics.f1_score(y_test, predictions)
    acc = metrics.accuracy_score(y_test, predictions)

    print(f"f1-score  is {f1}")
    print(f"acc  is {acc}")
    print(f"precision  is {precision}")
    print(f"recall  {recall}")
    print("Roc AUC:", roc_auc_score(y_test, predictions))

print('With no context')
print(scores(df["labels"], df["gpt_no_context"]))

print('\nWith context')
print(scores(df["labels"], df["gpt_with_context"]))